# PuLPモジュールを用いた最短路問題の最適解の計算
## Pandasを使用しない場合
## PuLPモジュール読み込み

In [1]:
import pulp

## 点の集合，枝の集合，目的関数の係数を定義
点（地点）の集合は点のリストとして定義する．
枝（地点間を結ぶ道路）は，枝(i,j)をタプル(i,j)で表し，そのリストとして枝の集合を定義する．
枝の重み（地点間の距離）は枝(i,j)をキーとする辞書として定義する．
始点sと終点tもここで定義しておく．

In [2]:
V = [i+1 for i in range(6)]
E = [(1,2), (1,3), (2,1), (2,3), (2,4), (3,1), (3,2), (3,5), (4,3), (4,6), (5,3), (5,6)]
W = {(1,2):1, (1,3):5, (2,1):1, (2,3):2, (2,4):2, (3,1):5, (3,2):4, (3,5):2, (4,3):3, (4,6):3, (5,3):1, (5,6):4}
s = 1
t = 5

## 決定変数の定義
枝をキーとする辞書として定義する．

In [3]:
x = {e:pulp.LpVariable(f'x{e[0]}_{e[1]}', cat=pulp.LpBinary) for e in E}

## 問題の定義
各決定変数に重み(距離)を乗じたものの和を目的関数として設定する．決定変数は枝に対応して定義されているので，総和を求めるにはpulp.lpSum()内で枝に関して繰り返す．

In [4]:
p = pulp.LpProblem('最短路問題', sense=pulp.LpMinimize)
p += pulp.lpSum(W[e]*x[e] for e in E), '目的関数　経路長'
for i in V:
    inflow = []
    outflow = []
    for e in E:
        if e[1] == i:
            inflow.append(e)
        if e[0] == i:
            outflow.append(e)
    net_outflow = pulp.lpSum(x[e] for e in outflow) - pulp.lpSum(x[e] for e in inflow)
    if i == s:
        p += net_outflow == 1, f'始点{i}'
    elif i == t:
        p += net_outflow == -1, f'終点{i}'
    else:
        p += net_outflow == 0, f'中間点{i}'
p

最短路問題:
MINIMIZE
1*x1_2 + 5*x1_3 + 1*x2_1 + 2*x2_3 + 2*x2_4 + 5*x3_1 + 4*x3_2 + 2*x3_5 + 3*x4_3 + 3*x4_6 + 1*x5_3 + 4*x5_6 + 0
SUBJECT TO
始点1: x1_2 + x1_3 - x2_1 - x3_1 = 1

中間点2: - x1_2 + x2_1 + x2_3 + x2_4 - x3_2 = 0

中間点3: - x1_3 - x2_3 + x3_1 + x3_2 + x3_5 - x4_3 - x5_3 = 0

中間点4: - x2_4 + x4_3 + x4_6 = 0

終点5: - x3_5 + x5_3 + x5_6 = -1

中間点6: - x4_6 - x5_6 = 0

VARIABLES
0 <= x1_2 <= 1 Integer
0 <= x1_3 <= 1 Integer
0 <= x2_1 <= 1 Integer
0 <= x2_3 <= 1 Integer
0 <= x2_4 <= 1 Integer
0 <= x3_1 <= 1 Integer
0 <= x3_2 <= 1 Integer
0 <= x3_5 <= 1 Integer
0 <= x4_3 <= 1 Integer
0 <= x4_6 <= 1 Integer
0 <= x5_3 <= 1 Integer
0 <= x5_6 <= 1 Integer

点iに入る枝の集合をinflow，出る枝の集合をoutflowとする．
各枝eを探索し，e[1] == i，すなわち枝eが(*,i)なら点iに入る枝なので，eをinflowに追加する．
inflow.append(e)はリストinflowにeを追加している．
e[0] == i，すなわち枝eが(i,*)なら点iから出る枝なので，eをoutflowに追加する．
inflowとoutflowが決まったら，出る枝に相当する変数の和から入る枝に相当する変数の和を引き，これをnet_outflowとする．
点iが始点sの場合は，net_outflowは1，点iが終点tの場合は，net_outflowは-1，点iが始点と終点以外の場合はnet_flowは0になる．

## 最適解の計算と計算結果の読み取り

In [5]:
result = p.solve()

In [6]:
pulp.LpStatus[result]

'Optimal'

In [7]:
pulp.value(p.objective)

5.0

In [8]:
for v in p.variables():
    if pulp.value(v) > 0:
        print(f'{v} = {pulp.value(v):.0f}')

x1_2 = 1
x2_3 = 1
x3_5 = 1


最適解は x1_2 = x2_3 = x3_5 = 1，すなわち1→2→3→5が最短路で，その距離は5．